# VDF Function Search For Edges

In [1]:
import qgrid
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [2]:
# Standard plotly imports
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

In [3]:
edge_volume_time = pd.read_pickle(r"..\Urban Freeway Dyn Assign Redmond.US\EdgeVolTime.pk1")
edge_attr = pd.read_pickle(r"..\Urban Freeway Dyn Assign Redmond.US\edges_attr.pk1")

In [4]:
qgrid.show_grid(edge_volume_time)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [18]:
list(edge_volume_time['Period'].unique())

[1, 2, 3, 4, 5]

In [5]:
qgrid.show_grid(edge_attr)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

## Create interactive travel time vs 15 min volume plot

In [19]:
@interact
def plot_VDF(edge=(1, edge_attr.index.max(), 1), vehtype=['ALL', '10'], mininterval=list(edge_volume_time['Period'].unique())):
    edge_vol_time_frame = edge_volume_time.loc[(edge_volume_time['NO'] == edge) & 
                                               (edge_volume_time['VehType'] == vehtype) &
                                               (edge_volume_time['Period'] >= mininterval), :]
    edge_vol_time_frame.iplot(x='VOLNEW',
                              y='TRAVTMNEW',
                              mode = 'markers',
                              xTitle='15 min volume',
                              yTitle='Edge travel time (mins)')

interactive(children=(IntSlider(value=664, description='edge', max=1327, min=1), Dropdown(description='vehtype…